In [69]:
import pandas as pd
import random
import re
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, PULP_CBC_CMD
from fpdf import FPDF

In [70]:
df = pd.read_csv("C:/Users/fzysndd/Desktop/24-25/sem2/TAOR/A level question bank - Sheet1.csv")

difficulty_map = {
    "Easy": 1,
    "Medium": 3,
    "High": 5
}
df["difficulty"] = df["difficulty"].replace(difficulty_map)
df["difficulty"] = pd.to_numeric(df["difficulty"], errors="coerce")
print(df["difficulty"].isna().sum())

df.to_csv("fixed_question_bank.csv", index=False)
print(df.head())


0
   Year Paper number  QN                        Type Type 2  score  difficulty
0  2018         8MA0  Q1                 Integration    NaN      4           1
1  2018         8MA0  Q2                  inequality    NaN      5           3
2  2018         8MA0  Q3  coordinate geometry&vector    NaN      5           1
3  2018         8MA0  Q4  coordinate geometry&vector    NaN      4           1
4  2018         8MA0  Q5    Exponents and Logarithms    NaN      5           3


In [ ]:
import numpy as np
random_weights = {i: np.random.uniform(0.9, 1.1) for i in df.index}
model += lpSum(x[i] * random_weights[i] for i in df.index)

model = LpProblem('Test_Paper_Selection', LpMinimize)

x = {i: LpVariable(f'x_{i}', 0, 1, cat='Binary') for i in df.index}

model += lpSum(x[i] for i in df.index) == 10

past_tests = [] 
max_repeats = 1

for past_test in past_tests:
    model += lpSum(x[i] for i in df.index if df.loc[i, "QN"] in past_test) <= max_repeats

knowledge_points = df["Type"].unique()
for kp in knowledge_points:
    model += lpSum(x[i] for i in df.index if df.loc[i, "Type"] == kp) >= 1

min_difficulty, max_difficulty = 1, 5
model += lpSum(df.loc[i, 'difficulty'] * x[i] for i in df.index) / 10 >= min_difficulty
model += lpSum(df.loc[i, 'difficulty'] * x[i] for i in df.index) / 10 <= max_difficulty

model += lpSum(df.loc[i, "score"] * x[i] for i in df.index) == 100

categories = df['Type'].unique()
category_constraints = {'Exponents and Logarithms': (0, 2), 'coordinate geometry&vector': (0, 2), 'Polynomials': (0, 2), 'Differentiation': (0, 2), 'Trigonometry': (0, 2), 'Integration': (0, 2), 'Proof / Reasoning / Written Explanations (Justifications, Inequality Counterexamples)': (0, 2), 'inequality': (0, 2)}
for cat in categories:
    if cat in category_constraints:
        min_c, max_c = category_constraints[cat]
        model += lpSum(x[i] for i in df.index if df.loc[i, 'Type'] == cat) >= min_c
        model += lpSum(x[i] for i in df.index if df.loc[i, 'Type'] == cat) <= max_c

model.solve(PULP_CBC_CMD(msg=False))

selected_questions = df.loc[[i for i in df.index if x[i].varValue == 1], ['Year', 'QN']]

selected_indices = [i for i in df.index if x[i].varValue == 1]

random.shuffle(selected_indices)
final_selection = selected_indices[:10]

selected_questions = df.loc[final_selection, ['Year', 'QN']]

print('select questions')
print(selected_questions)

#list form
selected_list = selected_questions.values.tolist()
print(selected_list)


select questions
     Year   QN
101  2021  Q12
182  2023  Q15
175  2023   Q8
114  2021   Q9
20   2018   Q6
105  2021  Q16
132  2022  Q12
108  2021   Q3
57   2019  Q13
[[2021, 'Q12'], [2023, 'Q15'], [2023, 'Q8'], [2021, 'Q9'], [2018, 'Q6'], [2021, 'Q16'], [2022, 'Q12'], [2021, 'Q3'], [2019, 'Q13']]
